In [10]:
import mne
import numpy as np
from scipy.signal import savgol_filter
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, accuracy_score
left_data = mne.io.read_raw_fif('cleaned_eeg_data_left2.fif', preload=True)
right_data = mne.io.read_raw_fif('cleaned_eeg_data_right.fif', preload=True)
up_data = mne.io.read_raw_fif('cleaned_eeg_data_up.fif', preload=True)
down_data = mne.io.read_raw_fif('cleaned_eeg_data_down.fif', preload=True)

epoch_duration = 2
overlap = 0.5  
left_events = mne.make_fixed_length_events(left_data, duration=epoch_duration)
right_events = mne.make_fixed_length_events(right_data, duration=epoch_duration)
up_events = mne.make_fixed_length_events(up_data, duration=epoch_duration)
down_events = mne.make_fixed_length_events(down_data, duration=epoch_duration)

left_epochs = mne.Epochs(left_data, events=left_events, tmin=0, tmax=epoch_duration, baseline=None, preload=True)
right_epochs = mne.Epochs(right_data, events=right_events, tmin=0, tmax=epoch_duration, baseline=None, preload=True)
up_epochs = mne.Epochs(up_data, events=up_events, tmin=0, tmax=epoch_duration, baseline=None, preload=True)
down_epochs = mne.Epochs(down_data, events=down_events, tmin=0, tmax=epoch_duration, baseline=None, preload=True)

left_epoch_test = left_epochs[5]
right_epoch_test = right_epochs[5]
up_epoch_test = up_epochs[5]
down_epoch_test = down_epochs[5]

left_epochs = left_epochs.drop(5)
right_epochs = right_epochs.drop(5)
up_epochs = up_epochs.drop(5)
down_epochs = down_epochs.drop(5)


Opening raw data file cleaned_eeg_data_left2.fif...
Isotrak not found
    Range : 0 ... 10527 =      0.000 ...    82.242 secs
Ready.
Reading 0 ... 10527  =      0.000 ...    82.242 secs...
Opening raw data file cleaned_eeg_data_right.fif...
Isotrak not found
    Range : 0 ... 16255 =      0.000 ...   126.992 secs
Ready.
Reading 0 ... 16255  =      0.000 ...   126.992 secs...
Opening raw data file cleaned_eeg_data_up.fif...
Isotrak not found
    Range : 0 ... 10136 =      0.000 ...    79.188 secs
Ready.
Reading 0 ... 10136  =      0.000 ...    79.188 secs...
Opening raw data file cleaned_eeg_data_down.fif...
Isotrak not found
    Range : 0 ... 18327 =      0.000 ...   143.180 secs
Ready.
Reading 0 ... 18327  =      0.000 ...   143.180 secs...
Not setting metadata
41 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 41 events and 257 original time points ...
0 bad epochs dropped
Not setting metadata
63 matching events foun

/tmp/ipykernel_313062/803537539.py:7: RuntimeWarning: This filename (cleaned_eeg_data_left2.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  left_data = mne.io.read_raw_fif('cleaned_eeg_data_left2.fif', preload=True)
/tmp/ipykernel_313062/803537539.py:8: RuntimeWarning: This filename (cleaned_eeg_data_right.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  right_data = mne.io.read_raw_fif('cleaned_eeg_data_right.fif', preload=True)
/tmp/ipykernel_313062/803537539.py:9: RuntimeWarning: This filename (cleaned_eeg_data_up.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_t

In [11]:


def preprocess_epochs(epochs_data, window_length=11, poly_order=2):
    preprocessed_epochs = []

    for epoch_data in epochs_data:
        processed_data = []
        for channel in range(epoch_data.shape[1]):
            smoothed_data = savgol_filter(epoch_data[:, channel], window_length, poly_order)
            processed_data.append(smoothed_data)

        big_channel = np.concatenate(processed_data)

        preprocessed_epochs.append(big_channel)

    return preprocessed_epochs





left_preprocessed = preprocess_epochs(left_epochs.get_data())
right_preprocessed = preprocess_epochs(right_epochs.get_data())
up_preprocessed = preprocess_epochs(up_epochs.get_data())
down_preprocessed = preprocess_epochs(down_epochs.get_data())


/tmp/ipykernel_313062/3149584071.py:20: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  left_preprocessed = preprocess_epochs(left_epochs.get_data())
/tmp/ipykernel_313062/3149584071.py:21: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  right_preprocessed = preprocess_epochs(right_epochs.get_data())
/tmp/ipykernel_313062/3149584071.py:22: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  up_preprocessed = preprocess_epochs(up_epochs.get_data())
/tmp/ipykernel_313062/3149584071.py:23: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  down_preprocessed = preprocess_epochs(down_epochs.get_data())


In [12]:


X = np.vstack((left_preprocessed, right_preprocessed, up_preprocessed, down_preprocessed))
labels = np.concatenate((np.zeros(len(left_preprocessed)),
                         np.ones(len(right_preprocessed)),
                         2 * np.ones(len(up_preprocessed)),
                         3 * np.ones(len(down_preprocessed))))


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

predicted_labels = lda.predict(X_test)

accuracy = accuracy_score(y_test, predicted_labels)
print(f"Accuracy on test set: {accuracy * 100:.2f}%")

report = classification_report(y_test, predicted_labels)
print("\nClassification Report on test set:")
print(report)



Accuracy on test set: 30.95%

Classification Report on test set:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         6
         1.0       0.18      0.17      0.17        12
         2.0       0.20      0.14      0.17         7
         3.0       0.43      0.59      0.50        17

    accuracy                           0.31        42
   macro avg       0.20      0.22      0.21        42
weighted avg       0.26      0.31      0.28        42



In [13]:
freq_bands = {'Delta': (0.5, 4),
              'Theta': (4, 8),
              'Alpha': (8, 13),
              'Beta': (13, 30),#should be more prevalent during our game
              'Gamma': (30, 40)}  

def compute_avg_band_amplitudes(epochs):
    avg_band_amplitudes = []
    for epoch in epochs:
        channel_band_amplitudes = []
        for channel_data in epoch:
            for band in freq_bands.values():
                fmin, fmax = band
                sp = np.fft.fft(channel_data)
                freq = np.fft.fftfreq(len(channel_data), d=1/left_data.info['sfreq'])
                freq = freq[1:int(np.ceil(len(channel_data) / 4))]  
                sp = sp[1:int(np.ceil(len(channel_data) / 4))]
                sp = np.sqrt(sp.real**2 + sp.imag**2)
                band_indices = np.logical_and(freq >= fmin, freq <= fmax)
                band_amplitude = np.mean(sp[band_indices])
                channel_band_amplitudes.append(band_amplitude)
        avg_band_amplitudes.append(np.mean(channel_band_amplitudes))
    return avg_band_amplitudes

left_avg_band_amplitudes = compute_avg_band_amplitudes(left_epochs)
right_avg_band_amplitudes = compute_avg_band_amplitudes(right_epochs)
up_avg_band_amplitudes = compute_avg_band_amplitudes(up_epochs)
down_avg_band_amplitudes = compute_avg_band_amplitudes(down_epochs)
all_avg_band_amplitudes = left_avg_band_amplitudes + up_avg_band_amplitudes+down_avg_band_amplitudes+right_avg_band_amplitudes

X2 = np.array(all_avg_band_amplitudes)
labels2 = np.concatenate((
    np.zeros(len(left_avg_band_amplitudes)),        # Left category labeled as 0
    np.ones(len(right_avg_band_amplitudes)),       # Right category labeled as 1
    np.full(len(up_avg_band_amplitudes), 2),       # Up category labeled as 2
    np.full(len(down_avg_band_amplitudes), 3)      # Down category labeled as 3
))


In [14]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, accuracy_score

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, labels2, test_size=0.2, random_state=42)
X_train2 = X_train2.reshape(X_train2.shape[0], -1)
X_test2 = X_test2.reshape(X_test2.shape[0], -1)

lda2 = LinearDiscriminantAnalysis()
lda2.fit(X_train2, y_train2)

predicted_labels2 = lda2.predict(X_test2)

accuracy2 = accuracy_score(y_test2, predicted_labels2)
print(f"Accuracy on test set for second model: {accuracy2 * 100:.2f}%")

report2 = classification_report(y_test2, predicted_labels2, zero_division=1) 
print("\nClassification Report on test set for second model:")
print(report2)


Accuracy on test set for second model: 47.62%

Classification Report on test set for second model:
              precision    recall  f1-score   support

         0.0       0.40      0.33      0.36         6
         1.0       0.50      0.42      0.45        12
         2.0       1.00      0.00      0.00         7
         3.0       0.48      0.76      0.59        17

    accuracy                           0.48        42
   macro avg       0.60      0.38      0.35        42
weighted avg       0.56      0.48      0.42        42



In [15]:
import mne
import numpy as np
from scipy.signal import savgol_filter
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

def preprocessRaw_epochs(epochs_data, window_length=11, poly_order=2):
    processedRaw = []
    min_length = min(len(epoch[0]) for epoch in epochs_data)

    for epoch_data in epochs_data:
        processed_data = []
        for channel_data in epoch_data:
            smoothed_data = savgol_filter(channel_data, window_length, poly_order)
            processed_data.append(smoothed_data[:min_length])

        processedRaw.append(np.array(processed_data))

    return processedRaw



processedRaw_left = preprocessRaw_epochs(left_epochs.get_data())
processedRaw_right = preprocessRaw_epochs(right_epochs.get_data())
processedRaw_up = preprocessRaw_epochs(up_epochs.get_data())
processedRaw_down = preprocessRaw_epochs(down_epochs.get_data())


X_processedRaw = np.vstack((processedRaw_left, processedRaw_right, processedRaw_up, processedRaw_down))
labels_processedRaw = np.concatenate((np.zeros(len(processedRaw_left)),
                                      np.ones(len(processedRaw_right)),
                                      2 * np.ones(len(processedRaw_up)),
                                      3 * np.ones(len(processedRaw_down))))



/tmp/ipykernel_313062/259963410.py:24: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  processedRaw_left = preprocessRaw_epochs(left_epochs.get_data())
/tmp/ipykernel_313062/259963410.py:25: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  processedRaw_right = preprocessRaw_epochs(right_epochs.get_data())
/tmp/ipykernel_313062/259963410.py:26: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  processedRaw_up = preprocessRaw_epochs(up_epochs.get_data())
/tmp/ipykernel_313062/259963410.py:27: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  processedRaw_down = preprocessRaw_epochs(down_epochs.get_data())


In [16]:

from sklearn.model_selection import train_test_split

X_processedRaw_flat = np.array([epoch.flatten() for epoch in X_processedRaw])

X_train_processedRaw_flat, X_test_processedRaw_flat, y_train_processedRaw, y_test_processedRaw = train_test_split(
    X_processedRaw_flat, labels_processedRaw, test_size=0.2, random_state=42)

lda3 = LinearDiscriminantAnalysis()
lda3.fit(X_train_processedRaw_flat, y_train_processedRaw)

predicted_labels_processedRaw = lda3.predict(X_test_processedRaw_flat)

accuracy_processedRaw = accuracy_score(y_test_processedRaw, predicted_labels_processedRaw)
print(f"Accuracy on processedRaw test set: {accuracy_processedRaw * 100:.2f}%")

report_processedRaw = classification_report(y_test_processedRaw, predicted_labels_processedRaw)
print("\nClassification Report on processedRaw test set:")
print(report_processedRaw)


Accuracy on processedRaw test set: 54.76%

Classification Report on processedRaw test set:
              precision    recall  f1-score   support

         0.0       0.33      0.17      0.22         6
         1.0       0.38      0.25      0.30        12
         2.0       0.44      0.57      0.50         7
         3.0       0.68      0.88      0.77        17

    accuracy                           0.55        42
   macro avg       0.46      0.47      0.45        42
weighted avg       0.50      0.55      0.51        42



In [17]:
#take eeg data
#apply ica
#get epochs(taking up_epoch_test)
up_test_preprocessed = preprocess_epochs(right_epoch_test.get_data())
up_epoch_test_np = np.array(up_test_preprocessed)
predicted_labels1 = lda.predict(up_epoch_test_np)
predicted_probabilities1 = lda.predict_proba(up_epoch_test_np)
max_probabilities1 = np.max(predicted_probabilities1, axis=1)



up_test_avg_band_amplitudes = compute_avg_band_amplitudes(up_epoch_test)
up_test_avg_band_amplitudes_np = np.array(up_test_avg_band_amplitudes)
up_test_avg_amplitude_reshaped = up_test_avg_band_amplitudes_np.reshape(up_test_avg_band_amplitudes_np.shape[0], -1)
predicted_probabilities_up_avg_amplitude = lda2.predict_proba(up_test_avg_amplitude_reshaped)
max_probabilities_up_avg_amplitude = np.max(predicted_probabilities_up_avg_amplitude, axis=1)
max_class_indices_up_avg_amplitude = np.argmax(predicted_probabilities_up_avg_amplitude, axis=1)


processedRaw_up_epoch_test = preprocessRaw_epochs(up_epoch_test.get_data())

X_processedRaw_up_epoch_test = np.array([epoch.flatten() for epoch in processedRaw_up_epoch_test])

predicted_labels_up_epoch_test = lda3.predict(X_processedRaw_up_epoch_test)
predicted_probabilities_up_epoch_test = lda3.predict_proba(X_processedRaw_up_epoch_test)

print(f"1st Model: Predicted Label - {predicted_labels1[0]}, Max Probability - {max_probabilities1[0]:.8f}")
print(f"2nd Model: Max Class - {max_class_indices_up_avg_amplitude[0]}, Max Probability - {max_probabilities_up_avg_amplitude[0]:.4f}")
print(f"3rd Model: Predicted Class - {predicted_labels_up_epoch_test[0]}, Max Probability - {np.max(predicted_probabilities_up_epoch_test[0]):.8f}")



1st Model: Predicted Label - 1.0, Max Probability - 0.74741166
2nd Model: Max Class - 3, Max Probability - 0.4037
3rd Model: Predicted Class - 1.0, Max Probability - 0.80053243


/tmp/ipykernel_313062/2118879545.py:4: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  up_test_preprocessed = preprocess_epochs(right_epoch_test.get_data())
/tmp/ipykernel_313062/2118879545.py:20: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  processedRaw_up_epoch_test = preprocessRaw_epochs(up_epoch_test.get_data())


In [18]:
def combined_prediction(acc1, acc2, acc3, prob1, prob2, prob3, class1, class2, class3):
    weighted_avg = (acc1 * prob1 * class1 + acc2 * prob2 * class2 + acc3 * prob3 * class3) / (acc1 + acc2 + acc3)
    return weighted_avg
acc1 = 0.3095  # Accuracy of model 1
acc2 = 0.4762  # Accuracy of model 2
acc3 = 0.5476  # Accuracy of model 3

prob1 = max_probabilities1[0]  
prob2 = max_probabilities_up_avg_amplitude[0]  
prob3 = np.max(predicted_probabilities_up_epoch_test[0])  

class1 = predicted_labels1[0] 
class2 = max_class_indices_up_avg_amplitude[0]  
class3 = predicted_labels_up_epoch_test[0]  

combined_pred = combined_prediction(acc1, acc2, acc3, prob1, prob2, prob3, class1, class2, class3)
combined_pred_label = round(combined_pred)  
print(f"Combined Prediction Label: {combined_pred_label}")

Combined Prediction Label: 1
